In [40]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

maxAngents = 100
arrivalRateMin = 1
arrivalRateMax = 8
service_xk = np.arange(6)+1
service_pk = (0.1, 0.2, 0.3, 0.25, 0.1, 0.05)
custm = stats.rv_discrete(name='custm', values=(service_xk, service_pk))

class EventsQueue:
    def __init__(self):
        self.globalTime = 0
        self.MEvents = []

    def QueueSize(self):
        return len(self.MEvents)

    def AddEvent(self,MEvent):
        count =  len(self.MEvents)
        if count == 0:
            self.MEvents.append(MEvent)
            return 0

        if(MEvent.eTime >= self.MEvents[count - 1].eTime ):
            self.MEvents.append(MEvent)
            return 0

        for i in range(0,count-1):
            if (MEvent.eTime >= self.MEvents[i].eTime):
                if (MEvent.eTime < self.MEvents[i + 1].eTime):
                    self.MEvents.insert(i + 1, MEvent)
                    return 0

    def ProcessNextEvent(self):
        if (len(self.MEvents) == 0):
            return 0
        self.MEvents[0].Execute()
        self.globalTime = self.MEvents[0].eTime
        del self.MEvents[0]

# Discrete Event System Specification
class DEVS:
    EQ = EventsQueue()
    GlobalTime = 0.0
    def __init__(self):
        pass

    @staticmethod
    def ProcessNextEvent():
        DEVS.EQ.ProcessNextEvent()
        DEVS.GlobalTime = DEVS.EQ.globalTime
# ---- Customer Statistics ----
class customerStat:
    def __init__(self):
         self.id = -1
         self.arrivalTime = -1
         self.serviceTime = -1
         self.interArrivalTime = 0
         self.serviceBegins = -1
         self.waitingTimeInQueue = 0
         self.serviceEnds = -1
         self.timeInSystem = -1
         self.idleTimeOfServer = 0

# ---- Arrival Event ----
class ArrivalEvent:
    def __init__(self):
         self.eTime = 0.0

    def Execute(self):
        customer = customerStat()
        customer.id = DEVS.newId
        customer.arrivalTime = self.eTime
        if(len(DEVS.stats)>0):
            customer.interArrivalTime = customer.arrivalTime - DEVS.stats[-1].arrivalTime

        # print("Time %d"%self.eTime," Arrival Event of agent {0}".format(customer.id))
        if(DEVS.newId<maxAngents-1):
            NextArrival = ArrivalEvent()
            NextArrival.eTime = self.eTime+random.randint(arrivalRateMin,arrivalRateMax)
            DEVS.EQ.AddEvent(NextArrival)

        # server is Free
        if (DEVS.serverIdle == True):
            DEVS.serverIdle = False
            # print("server is Busy")
            Service = ServiceEvent()
            serviceTime = custm.rvs()
            customer.serviceTime = serviceTime
            customer.serviceBegins = self.eTime # current time
            Service.eTime = self.eTime+serviceTime
            Service.id = customer.id
            DEVS.EQ.AddEvent(Service)
        # server is Busy
        else:
            # increase waiting line
            DEVS.customerQueue.append(customer.id)
            # print("customerQueue = %d"%len(DEVS.customerQueue))

        DEVS.newId = DEVS.newId + 1
        DEVS.stats.append(customer)

# ---- Service (END) Event ----
class ServiceEvent:
    def __init__(self):
         self.eTime = 0.0
         self.id = 0

    def Execute(self):
        ind = [i for i,val in enumerate(DEVS.stats) if val.id == self.id][0]
        DEVS.stats[ind].serviceEnds = self.eTime
        DEVS.stats[ind].timeInSystem = DEVS.stats[ind].serviceEnds - DEVS.stats[ind].arrivalTime
        DEVS.stats[ind].waitingTimeInQueue = DEVS.stats[ind].serviceBegins - DEVS.stats[ind].arrivalTime # 0 without queue
        DEVS.stats[ind].idleTimeOfServer = DEVS.stats[ind].serviceBegins - DEVS.lastServedTime

        # print("Time %d"%self.eTime,"Service finished")
        if(len(DEVS.customerQueue)>0):
            qid = DEVS.customerQueue.pop(0)
            qind = [i for i,val in enumerate(DEVS.stats) if val.id == qid][0]
            Service = ServiceEvent()
            serviceTime = custm.rvs()
            Service.eTime = self.eTime+serviceTime
            Service.id = qid
            DEVS.stats[qind].serviceBegins = self.eTime
            DEVS.stats[qind].serviceTime = serviceTime
            DEVS.EQ.AddEvent(Service)
            # print("take new customer from the queue")
        else:
            DEVS.serverIdle = True
            # print("server is Idle (do nothing)")

        DEVS.lastServedTime = self.eTime

def get_simulated_value():
   AE = ArrivalEvent()
   DEVS.EQ.AddEvent(AE)

   # simulation attributes
   DEVS.customerQueue = []
   DEVS.stats = []
   DEVS.newId = 0
   DEVS.serverIdle = True
   DEVS.lastServedTime = 0 # for Idle time

   # --- SIMULATION ---
   while DEVS.EQ.QueueSize()>0:
       DEVS.ProcessNextEvent()
   stats = DEVS.stats

   av_time_in_system = sum([x.timeInSystem for x in stats]) / len(stats)
   return av_time_in_system


def get_mean_and_estimation(data, log=True):
    Y = data.mean()
    S = np.sqrt(data.var(ddof=1))
    R = data.size
    t = stats.t.ppf(0.975, R - 1)
    H = t * S / np.sqrt(R)
    if(log):
       print('\nТекущие значения среднего и оценки', Y, '±', H)
    return H, t, S, R

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = a.size
    se = np.std(a, ddof=1) / np.sqrt(n)
    t = stats.t._ppf((1 + confidence) / 2., n - 1)
    h = se * t
    return a.mean(), h

def get_needed_runs(epsilon, data):
    H, t, S, R = get_mean_and_estimation(data,False)
    runs = 1
    while H > epsilon:
        runs += 1
        H = t * S / np.sqrt(R + runs)
    return runs


def check_estimation(runs, data):
    for i in range(runs):
        data = np.append(data, get_simulated_value())
        if mean_confidence_interval(data)[1] < epsilon:
            get_mean_and_estimation(data)
            return i + 1
    get_mean_and_estimation(data)
    return data


total_runs = 5
epsilon = 0.1
print('Параметр: Среднее время в системе.')

data = np.array([])
for i in range(total_runs):
    data = np.append(data, get_simulated_value())

get_mean_and_estimation(data)
runs = get_needed_runs(epsilon, data)

print("Ожидаемое количество репликаций:", total_runs+runs)
# проверим первоначальную оценку
while mean_confidence_interval(data)[1] > epsilon:
    data = check_estimation(runs, data)
    if not isinstance(data, int):
        print('Точность не достигнута, keep rollin')
        total_runs += runs
        runs = get_needed_runs(epsilon, data)
        print("Ожидаемое количество репликаций:", total_runs+runs)
    else:
        total_runs += data
print('Всего запусков было произведено', total_runs)

Параметр: Среднее время в системе.

Текущие значения среднего и оценки 4.422 ± 0.5751598868795492
Ожидаемое количество репликаций: 166

Текущие значения среднего и оценки 4.522469879518073 ± 0.10113247038096626
Точность не достигнута, keep rollin
Ожидаемое количество репликаций: 170

Текущие значения среднего и оценки 4.522662721893491 ± 0.09956918632800288
Всего запусков было произведено 169
